In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense

# Load the dataset from your PC
# Assuming your dataset is in CSV format with 'review' and 'sentiment' columns
data = pd.read_csv('IMDB.csv')

# Convert sentiment labels to numerical form
data['sentiment'] = data['sentiment'].map({'positive': 1, 'negative': 0})

# Separate features and labels
reviews = data['review'].values
sentiments = data['sentiment'].values

# Tokenize the text
tokenizer = Tokenizer(num_words=10000)
tokenizer.fit_on_texts(reviews)
sequences = tokenizer.texts_to_sequences(reviews)

# Pad sequences to ensure uniform length
padded_sequences = pad_sequences(sequences, maxlen=500)

# Split the data into train and test sets
x_train, x_test, y_train, y_test = train_test_split(padded_sequences, sentiments, test_size=0.2, random_state=42)

# Define the model
model = Sequential([
    Embedding(10000, 128),
    LSTM(128, dropout=0.2, recurrent_dropout=0.2),
    Dense(1, activation='sigmoid')
])

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(x_train, y_train, batch_size=32, epochs=5, validation_data=(x_test, y_test))

# Function to preprocess input text
def preprocess_text(text):
    sequence = tokenizer.texts_to_sequences([text])
    padded_sequence = pad_sequences(sequence, maxlen=500)
    return padded_sequence

# Function to predict sentiment
def predict_sentiment(text):
    # Preprocess the input text
    sample_sequence = preprocess_text(text)
    # Predict
    prediction = model.predict(sample_sequence)
    return prediction[0][0]

# Sample text
sample_text = "This movie is fantastic! I loved every moment of it."

# Predict sentiment
prediction = predict_sentiment(sample_text)
print("Predicted sentiment (0: Negative, 1: Positive):", prediction)
